In [1]:
import requests, json, io
from PIL import Image

In [2]:
wop = json.load(open('/home/nika/Downloads/default-cards-20240718211147.json', 'rb'))
#wop = json.load(open('C:/Users/Nika/Downloads/default-cards-20240715090650.json', 'rb'))
print(len(wop), 'entries')
print('{:.1f} mb'.format(sum([len(json.dumps(i)) for i in wop]) / 1000 / 1000))

96744 entries
472.0 mb


In [3]:
def Flagger(jsonEntries):
    jsonFlagged = []
    skipped = {
        'digitalOnly' : 0,
        'badLayout' : 0,
        'badPromotype' : 0
    }
    for jsonEntry in jsonEntries:
        if 'paper' not in jsonEntry['games']:
            skipped['digitalOnly'] += 1
            continue
        if jsonEntry['layout'] in ['art_series', 'double_faced_token', 'token', 'planar', 'scheme', 'host', 'vanguard', 'emblem']:
            skipped['badLayout'] += 1
            continue
        if 'promo_types' in jsonEntry:
            if 'thick' in jsonEntry['promo_types']:
                skipped['badPromotype'] += 1
                continue
        jsonFlagged.append(jsonEntry)
    return jsonFlagged

flagged = Flagger(wop)
print(len(flagged), 'entries')
print('{:.1f} mb'.format(sum([len(json.dumps(i)) for i in flagged]) / 1000 / 1000))

83429 entries
391.9 mb


In [4]:
class Face:
    def __init__(self, card):
        self.Name = card['name']
        self.ManaCost = card['mana_cost']
        self.TypeLine = card['type_line']
        self.OracleText = card['oracle_text']
        if 'power' in card: self.Power = card['power']
        else: self.Power = None
        if 'toughness' in card: self.Toughness = card['toughness']
        else: self.Toughness = None
        if 'defense' in card: self.Defense = card['defense']
        else: self.Defense = None
            
    
class Variant:
    FlavorText = None
    FlavorName = None
    def __init__(self, card):
        self.SetName = card['set_name']
        self.SetCode = card['set']
        self.CollNum = card['collector_number']
        self.Finishes = card['finishes']
        self.Artist = card['artist']
        self.Released = card['released_at']
        self.Rarity = card['rarity']
        if 'image_uris' in card:
            self.Thumb = [card['image_uris']['small']]
            self.Image = [card['image_uris']['normal']]
            if 'flavor_text' in card: self.FlavorText = card['flavor_text']
            if 'flavor_name' in card: self.FlavorName = card['flavor_name']
        else:
            self.Thumb = [i['image_uris']['small'] for i in card['card_faces']]
            self.Image = [i['image_uris']['normal'] for i in card['card_faces']]
            if any('flavor_text' in i for i in card['card_faces']):
                self.FlavorText = [i['flavor_text'] if 'flavor_text' in i else '' for i in card['card_faces']]
                self.FlavorName = [i['flavor_name'] if 'flavor_name' in i else '' for i in card['card_faces']]


class OracleCard:
    def __init__(self, card):
        if 'oracle_id' in card: self.OracleId = card['oracle_id']
        else: self.OracleId = card['card_faces'][0]['oracle_id']
        self.Name = card['name']
        if 'card_faces' in card: self.Faces = [Face(i) for i in card['card_faces']]
        else: self.Faces = [Face(card)]
        self.ColorIdentity = card['color_identity']
        if 'cmc' in card: self.CMC = card['cmc']
        else: self.CMC = card['card_faces'][0]['cmc']
        self.Variants = [Variant(card)]
        self.Keywords = card['keywords']
        if 'type_line' in card: self.TypeLine = card['type_line']
        else: self.TypeLine = card['card_faces'][0]['type_line']
        self.Legalities = card['legalities']
    def __repr__(self):
        return json.dumps({
            'Name' : self.Name,
            'Faces' : [i.__dict__ for i in self.Faces],
            'ColorIdentity' : self.ColorIdentity,
            'CMC' : self.CMC,
            'Variants' : [i.__dict__ for i in self.Variants],
            'Keywords' : self.Keywords,
            'TypeLine' : self.TypeLine,
            'Legalities' : self.Legalities
        })

        
database = {}
for i in flagged:
    if 'oracle_id' in i: _oracleId = i['oracle_id']
    else: _oracleId = i['card_faces'][0]['oracle_id']
    if _oracleId in database:
        database[_oracleId].Variants.append(Variant(i))
    else:
        database[_oracleId] = OracleCard(i)
        
print(len(database), 'entries')
print('{:.1f} mb'.format(sum([len(str(database[i])) for i in database]) / 1000 / 1000))

28699 entries
66.2 mb


In [5]:
[database[i] for i in database if 'Sasaya' in database[i].Name]

[{"Name": "Sasaya, Orochi Ascendant // Sasaya's Essence", "Faces": [{"Name": "Sasaya, Orochi Ascendant", "ManaCost": "{1}{G}{G}", "TypeLine": "Legendary Creature \u2014 Snake Monk", "OracleText": "Reveal your hand: If you have seven or more land cards in your hand, flip Sasaya, Orochi Ascendant.", "Power": "2", "Toughness": "3", "Defense": null}, {"Name": "Sasaya's Essence", "ManaCost": "", "TypeLine": "Legendary Enchantment", "OracleText": "Whenever a land you control is tapped for mana, add an additional one mana of any type that land produced for each other land you control with the same name as it.", "Power": null, "Toughness": null, "Defense": null}], "ColorIdentity": ["G"], "CMC": 3.0, "Variants": [{"SetName": "Saviors of Kamigawa", "SetCode": "sok", "CollNum": "145", "Finishes": ["nonfoil", "foil"], "Artist": "Christopher Moeller", "Released": "2005-06-03", "Rarity": "rare", "Thumb": ["https://cards.scryfall.io/small/front/d/2/d224c50f-8146-4c91-9401-04e5bd306d02.jpg?1562496100"

In [67]:
[i for i in flagged if 'image_uris' not in i and i['card_faces'][0]['artist'] != i['card_faces'][1]['artist']][1]

{'object': 'card',
 'id': '1f6213d9-c65d-48da-a501-279db5f31d6a',
 'oracle_id': '0b55eac6-a745-4bf4-8926-5ce83bc38d7d',
 'multiverse_ids': [],
 'tcgplayer_id': 151718,
 'cardmarket_id': 313036,
 'name': 'Treasure Map // Treasure Cove',
 'lang': 'en',
 'released_at': '2017-11-24',
 'uri': 'https://api.scryfall.com/cards/1f6213d9-c65d-48da-a501-279db5f31d6a',
 'scryfall_uri': 'https://scryfall.com/card/pxtc/250/treasure-map-treasure-cove?utm_source=api',
 'layout': 'transform',
 'highres_image': True,
 'image_status': 'highres_scan',
 'cmc': 2.0,
 'type_line': 'Artifact // Land',
 'color_identity': [],
 'keywords': ['Transform', 'Treasure', 'Scry'],
 'produced_mana': ['B', 'C', 'G', 'R', 'U', 'W'],
 'card_faces': [{'object': 'card_face',
   'name': 'Treasure Map',
   'mana_cost': '{2}',
   'type_line': 'Artifact',
   'oracle_text': '{1}, {T}: Scry 1. Put a landmark counter on Treasure Map. Then if there are three or more landmark counters on it, remove those counters, transform Treasure 

In [64]:
[i for i in flagged if 'card_faces' in i and 'flavor_name' in i['card_faces'][0]][0]

{'object': 'card',
 'id': '0954ed3d-a5d8-4330-b07c-bf3e51192e4b',
 'oracle_id': 'a720f3d3-c2d7-4e50-8cc6-600120380e9c',
 'multiverse_ids': [544475, 544476],
 'arena_id': 79141,
 'tcgplayer_id': 253196,
 'cardmarket_id': 581328,
 'name': 'Voldaren Bloodcaster // Bloodbat Summoner',
 'lang': 'en',
 'released_at': '2021-11-19',
 'uri': 'https://api.scryfall.com/cards/0954ed3d-a5d8-4330-b07c-bf3e51192e4b',
 'scryfall_uri': 'https://scryfall.com/card/vow/338/voldaren-bloodcaster-bloodbat-summoner?utm_source=api',
 'layout': 'transform',
 'highres_image': True,
 'image_status': 'highres_scan',
 'cmc': 2.0,
 'type_line': 'Creature — Vampire Wizard // Creature — Vampire Wizard',
 'color_identity': ['B'],
 'keywords': ['Flying', 'Transform'],
 'card_faces': [{'object': 'card_face',
   'name': 'Voldaren Bloodcaster',
   'flavor_name': 'Dracula, Lord of Blood',
   'mana_cost': '{1}{B}',
   'type_line': 'Creature — Vampire Wizard',
   'oracle_text': 'Flying\nWhenever Voldaren Bloodcaster or anothe

In [73]:
[i['name'] for i in flagged if 'type_line' not in i]

['Ghalta, Primal Hunger // Ghalta, Primal Hunger',
 'Stitch in Time // Stitch in Time',
 'Anointed Procession // Anointed Procession',
 'Jetmir, Nexus of Revels // Jetmir, Nexus of Revels',
 'Jace Beleren // Jace Beleren',
 'Norin the Wary // Norin the Wary',
 'Propaganda // Propaganda',
 'Jace Beleren // Jace Beleren',
 'Blightsteel Colossus // Blightsteel Colossus',
 'Chandra Nalaar // Chandra Nalaar',
 'Terror // Terror',
 'Plains // Plains',
 'Krark, the Thumbless // Krark, the Thumbless',
 'Adrix and Nev, Twincasters // Adrix and Nev, Twincasters',
 'Unholy Grotto // Unholy Grotto',
 'Island // Island',
 'Forest // Forest',
 'Ulamog, the Ceaseless Hunger // Ulamog, the Ceaseless Hunger',
 'Swamp // Swamp',
 'Chandra Nalaar // Chandra Nalaar',
 'Darksteel Colossus // Darksteel Colossus',
 'Zombie Master // Zombie Master',
 'Doubling Cube // Doubling Cube',
 'Liliana Vess // Liliana Vess',
 'Okaun, Eye of Chaos // Okaun, Eye of Chaos',
 'Ajani Goldmane // Ajani Goldmane',
 'Noxious 

In [42]:
[i for i in flagged if 'card_faces' in i][1]

{'object': 'card',
 'id': '00255899-aaaf-46c6-8037-bd0e3c06250c',
 'oracle_id': '7f8dad7f-a573-469f-9095-4ba3886b4f0b',
 'multiverse_ids': [607310, 607311],
 'mtgo_id': 110392,
 'arena_id': 84546,
 'tcgplayer_id': 490498,
 'cardmarket_id': 703462,
 'name': 'Invasion of Tolvada // The Broken Sky',
 'lang': 'en',
 'released_at': '2023-04-21',
 'uri': 'https://api.scryfall.com/cards/00255899-aaaf-46c6-8037-bd0e3c06250c',
 'scryfall_uri': 'https://scryfall.com/card/mom/241/invasion-of-tolvada-the-broken-sky?utm_source=api',
 'layout': 'transform',
 'highres_image': True,
 'image_status': 'highres_scan',
 'cmc': 5.0,
 'type_line': 'Battle — Siege // Enchantment',
 'color_identity': ['B', 'W'],
 'keywords': ['Transform'],
 'card_faces': [{'object': 'card_face',
   'name': 'Invasion of Tolvada',
   'mana_cost': '{3}{W}{B}',
   'type_line': 'Battle — Siege',
   'oracle_text': "(As a Siege enters, choose an opponent to protect it. You and others can attack it. When it's defeated, exile it, then

In [46]:
def Parser(jsonEntries):
    database = {}
    skipped = {
        'missingID' : 0,
        'digitalOnly' : 0,
        'badLayout' : 0,
        'badPromotype' : 0,
        'missingIMG' : 0,
        'other' : 0
    }
    for jsonEntry in jsonEntries:
#         if 'oracle_id' not in jsonEntry:
#             skipped['missingID'] += 1
#             continue
        if 'paper' not in jsonEntry['games']:
            skipped['digitalOnly'] += 1
            continue
        if jsonEntry['layout'] in ['art_series', 'double_faced_token', 'token', 'planar', 'scheme', 'host', 'vanguard', 'emblem']:
            skipped['badLayout'] += 1
            continue
        if 'promo_types' in jsonEntry:
            if 'thick' in jsonEntry['promo_types']:
                skipped['badPromotype'] += 1
                continue
        if 'image_uris' not in jsonEntry: # transform, modal_dfc, reversible_card
            1 # store face as main object, store back in FlipSide att., how to adopt Variants att.?
        try:
            if 'oracle_id' not in jsonEntry: # these are cards with flip arts so id[0] == id[1]
                jsonEntry['oracle_id'] = jsonEntry['card_faces'][0]['oracle_id']
            for i in ['flavor_name', 'flavor_text']:
                if i not in jsonEntry:
                    jsonEntry[i] = ""
            if jsonEntry['oracle_id'] not in database:
                database[jsonEntry['oracle_id']] = {
                    'OracleId' : jsonEntry['oracle_id'],
                    'Name' : jsonEntry['name'],
                    'Variants' : [],
                    'TypeLine' : jsonEntry['type_line'],
                    'OracleText' : jsonEntry['oracle_text'],
                    'ManaCost' : jsonEntry['mana_cost'],
                    'CMC' : jsonEntry['cmc'],
                    'Colors' : jsonEntry['colors'],
                    'ColorIdentity' : jsonEntry['color_identity'],
                    'Keywords' : jsonEntry['keywords'],
                    'Legalities' : jsonEntry['legalities']
                }
            database[jsonEntry['oracle_id']]['Variants'] += [{
                'SetName' : jsonEntry['set_name'],
                'SetCode' : jsonEntry['set'],
                'CollNum' : jsonEntry['collector_number'],
                'Finishes' : jsonEntry['finishes'],
                'Artist' : jsonEntry['artist'],
                'Thumb' : jsonEntry['image_uris']['small'],
                'Image' : jsonEntry['image_uris']['normal'],
                'Language' : jsonEntry['lang'],
                'Released' : jsonEntry['released_at'],
                'Rarity' : jsonEntry['rarity'],
                'FlavorName' : jsonEntry['flavor_name'],
                'FlavorText' : jsonEntry['flavor_text']
            }]
        except:
            skipped['other'] += 1
            pass
    return skipped, database

skipped, database = Parser(wop)
        
print('Skipped', skipped, 'entries')
print(len(database), 'entries')

keys = database.keys()
dbsize = 0
for key in keys:
    dbsize += len(json.dumps(database[key]))
    
print('{:.2f} mb'.format(dbsize / 1000 / 1000))

Skipped {'missingID': 0, 'digitalOnly': 8359, 'badLayout': 4867, 'badPromotype': 89, 'missingIMG': 0, 'other': 1583} entries
28019 entries
66.02 mb


In [41]:
sum(['image_uris' not in i and 'card_faces' not in i for i in wop])

0

In [34]:
sum(['card_faces' in i for i in wop])

3735

In [44]:
set([i['layout'] for i in wop if 'image_uris' not in i and 'card_faces' in i])

{'art_series',
 'double_faced_token',
 'modal_dfc',
 'reversible_card',
 'transform'}

In [17]:
[oracleDatabase[i] for i in oracleDatabase if 'Wojek Investigator' in oracleDatabase[i]['Name']]

[{'OracleId': 'ab41243d-b178-4ebe-a7ac-bea37427be99',
  'Name': 'Wojek Investigator',
  'Variants': [],
  'TypeLine': 'Creature — Angel Detective',
  'OracleText': 'Flying, vigilance\nAt the beginning of your upkeep, investigate once for each opponent who has more cards in hand than you. (To investigate, create a Clue token. It\'s an artifact with "{2}, Sacrifice this artifact: Draw a card.")',
  'ManaCost': '{2}{W}',
  'CMC': 3.0,
  'Colors': ['W'],
  'ColorIdentity': ['W'],
  'Keywords': ['Flying', 'Investigate', 'Vigilance'],
  'Legalities': {'standard': 'legal',
   'future': 'legal',
   'historic': 'legal',
   'timeless': 'legal',
   'gladiator': 'legal',
   'pioneer': 'legal',
   'explorer': 'legal',
   'modern': 'legal',
   'legacy': 'legal',
   'pauper': 'not_legal',
   'vintage': 'legal',
   'penny': 'legal',
   'commander': 'legal',
   'oathbreaker': 'legal',
   'standardbrawl': 'legal',
   'brawl': 'legal',
   'alchemy': 'legal',
   'paupercommander': 'not_legal',
   'duel': 

In [37]:
scryfallUri = "https://api.scryfall.com/"

def Search(searchValue):
    searchUri = ""
    for char in searchValue.lower():
        if char in "abcdefghijklmnopqrstuvwxyz0123456789 ":
            searchUri += char
    searchUri = searchUri.replace(' ', '+')
    resp = requests.get(scryfallUri + "cards/named?fuzzy=" + searchUri)
    return json.loads(resp.text)

def Variants(cardObject):
    oracleId = cardObject['oracle_id']
    resp = requests.get(scryfallUri + "cards/search?order=released&unique=prints&q=game%3Apaper+oracleid%3A" + oracleId)
    vdat = json.loads(resp.text)['data']
    vlst = []
    for i in vdat:
        cdat = {
            'finishes' : i['finishes'],
            'scryfall' : i['id'],
            'thumb' : i['image_uris']['small'],
            'image' : i['image_uris']['normal']
        }
        vlst += [["{} ({}) {}".format(i['set_name'], i['set'], i['collector_number']), cdat]]
    return vlst

In [53]:
wok = Variants(Search('wojek invest'))
wok

[['Murders at Karlov Manor Promos (pmkm) 36s',
  {'finishes': ['foil'],
   'scryfall': '97b2df60-f773-4b61-9101-05642a227cf5',
   'thumb': 'https://cards.scryfall.io/small/front/9/7/97b2df60-f773-4b61-9101-05642a227cf5.jpg?1707551491',
   'image': 'https://cards.scryfall.io/normal/front/9/7/97b2df60-f773-4b61-9101-05642a227cf5.jpg?1707551491'}],
 ['Murders at Karlov Manor Promos (pmkm) 36p',
  {'finishes': ['nonfoil', 'foil'],
   'scryfall': '091748a0-6a01-40cb-9ab8-015aa8235d3c',
   'thumb': 'https://cards.scryfall.io/small/front/0/9/091748a0-6a01-40cb-9ab8-015aa8235d3c.jpg?1707510568',
   'image': 'https://cards.scryfall.io/normal/front/0/9/091748a0-6a01-40cb-9ab8-015aa8235d3c.jpg?1707510568'}],
 ['Murders at Karlov Manor (mkm) 429',
  {'finishes': ['foil'],
   'scryfall': '58eccc85-bdfe-48f4-a79e-76d9ef122815',
   'thumb': 'https://cards.scryfall.io/small/front/5/8/58eccc85-bdfe-48f4-a79e-76d9ef122815.jpg?1712996935',
   'image': 'https://cards.scryfall.io/normal/front/5/8/58eccc85-

In [51]:
[Image.open(io.BytesIO(requests.get(i[1]['thumb']).content)) for i in wok]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>]

In [38]:
offlineKeys = {
    'object' : True,
    'id': True,
    'oracle_id' : True,
    'multiverse_ids' : False,
    'mtgo_id' : False,
    'mtgo_foil_id' : False,
    'tcgplayer_id' : False,
    'cardmarket_id' : False,
    'name' : True,
    'lang' : True,
    'released_at' : True,
    'uri' : False,
    'scryfall_uri' : False,
    'layout' : True,
    'highres_image' : False,
    'image_status' : False,
    'image_uris' : False,
    'mana_cost' : True,
    'cmc' : True,
    'type_line' : True,
    'oracle_text' : True,
    'power' : True,
    'toughness' : True,
    'colors' : True,
    'color_identity' : True,
    'keywords' : True,
    'all_parts' : False,
    'legalities' : True,
    'games' : False,
    'reserved' : False,
    'foil' : False,
    'nonfoil' : False,
    'finishes' : False,
    'oversized' : False,
    'promo' : False,
    'reprint' : False,
    'variation' : False,
    'set_id' : False,
    'set_name' : True,
    'set_type' : False,
    'set_uri' : False,
    'set_search_uri' : False,
    'scryfall_set_uri' : False,
    'rulings_uri' : False,
    'prints_search_uri' : False,
    'collector_number' : True,
    'digital' : False,
    'rarity' : True,
    'card_back_id' : False,
    'artist' : True,
    'artist_ids' : False,
    'illustration_id' : False,
    'border_color' : False,
    'frame' : False,
    'security_stamp' : False,
    'full_art' : False,
    'textless' : False,
    'booster' : False,
    'story_spotlight' : False,
    'edhrec_rank' : False,
    'penny_rank' : False,
    'prices' : False,
    'related_uris' : False,
    'purchase_uris' : False
}

In [39]:
fullJSON = json.loads(resp.content)

selectKeys = [i for i in offlineKeys if offlineKeys[i]]
offlineJSON = { i : fullJSON[i] for i in selectKeys}

In [40]:
offlineJSON

{'object': 'card',
 'id': '6af12a34-52a1-4940-a8d5-9e9811fcd59f',
 'oracle_id': 'e245d5c9-ae3a-4246-bd74-c6fed76b9fad',
 'name': 'Temmet, Vizier of Naktamun',
 'lang': 'en',
 'released_at': '2017-04-28',
 'layout': 'normal',
 'mana_cost': '{W}{U}',
 'cmc': 2.0,
 'type_line': 'Legendary Creature — Human Cleric',
 'oracle_text': "At the beginning of combat on your turn, target creature token you control gets +1/+1 until end of turn and can't be blocked this turn.\nEmbalm {3}{W}{U} ({3}{W}{U}, Exile this card from your graveyard: Create a token that's a copy of it, except it's a white Zombie Human Cleric with no mana cost. Embalm only as a sorcery.)",
 'power': '2',
 'toughness': '2',
 'colors': ['U', 'W'],
 'color_identity': ['U', 'W'],
 'keywords': ['Embalm'],
 'legalities': {'standard': 'not_legal',
  'future': 'not_legal',
  'historic': 'legal',
  'timeless': 'legal',
  'gladiator': 'legal',
  'pioneer': 'legal',
  'explorer': 'legal',
  'modern': 'legal',
  'legacy': 'legal',
  'paup

In [41]:
len(resp.content), len(json.dumps(offlineJSON))

(5151, 1366)

In [42]:
numberCardsEDH = 15103

print('Full JSON size: {:5.1f} mb'.format(numberCardsEDH * len(resp.content) / 1000 / 1000))
print('Offl JSON size: {:5.1f} mb'.format(numberCardsEDH * len(json.dumps(offlineJSON)) / 1000 / 1000))
print('Imgs JSON size: {:5.1f} mb'.format(numberCardsEDH * len(requests.get(json.loads(resp.text)['image_uris']['small']).content) / 1000 / 1000))

Full JSON size:  77.8 mb
Offl JSON size:  20.6 mb
Imgs JSON size: 221.6 mb
